# ハンズオンラボ：Databricksでエージェントシステムを構築する

## パート2 - エージェント評価
エージェントを作成したので、そのパフォーマンスをどのように評価するのでしょうか？
第2部では、評価に焦点を当てるために製品サポートエージェントを作成します。
このエージェントは、RAGアプローチを使用して製品ドキュメントを活用し、製品に関する質問に回答します。

### 2.1 新しいエージェントとリトリーバーツールの定義
- [**agent.py**]($./agent.py)：サンプルエージェントが設定されています。まずこのファイルを確認し、構成要素を理解しましょう
- **ベクター検索**：特定の製品に関連するドキュメントを検索できるベクター検索エンドポイントを作成しました。
- **リトリーバー関数の作成**：リトリーバーのプロパティを定義し、LLMから呼び出せるようにパッケージ化します。

### 2.2 評価データセットの作成
- サンプルの評価データセットを用意していますが、[合成的に生成](https://www.databricks.com/jp/blog/streamline-ai-agent-evaluation-with-new-synthetic-data-capabilities)することも可能です。

### 2.3 MLflow.evaluate() の実行
- MLflowは評価データセットを使ってエージェントの応答をテストします
- LLMジャッジが出力をスコア化し、すべてを見やすいUIでまとめます

### 2.4 必要な改善を行い、再評価を実施
- 評価結果からフィードバックを得てリトリーバー設定を変更
- 再度評価を実施し、改善を確認しましょう！

In [0]:
%pip install -U -qqqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

In [0]:
%run ../config

## 環境変数を通じたエージェントの設定

環境変数から`agent.py`のパラメータを設定します。

In [0]:
from databricks.sdk import WorkspaceClient
import os
import re

# ワークスペースクライアントを使用して現在のユーザーに関する情報を取得
w = WorkspaceClient()
user_email = w.current_user.me().emails[0].value
username = user_email.split('@')[0]
username = re.sub(r'[^a-zA-Z0-9_]', '_', username) # 特殊文字をアンダースコアに置換

# スキーマを指定します
user_schema_name = f"agents_lab_{username}" # ユーザーごとのスキーマ

In [0]:
# LLMエンドポイント名
os.environ["LLM_ENDPOINT_NAME"] = "databricks-llama-4-maverick"

# UC関数ツール
os.environ["UC_TOOL_NAMES"] = f"{catalog_name}.{user_schema_name}.*"

# Vector Search名
os.environ["VS_NAME"] = f"{catalog_name}.{system_schema_name}.product_docs_index"

print("環境変数を設定しました:")
print(f"LLM_ENDPOINT_NAME: {os.environ.get('LLM_ENDPOINT_NAME')}")
print(f"UC_TOOL_NAMES: {os.environ.get('UC_TOOL_NAMES')}")
print(f"VS_NAME: {os.environ.get('VS_NAME')}")

In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "Soundwave X5 Pro ヘッドフォンのトラブルシューティングのコツを教えてください。"}]})

In [0]:
AGENT.predict({"messages": [{"role": "user", "content": "今日の日付は"}]})

In [0]:
from IPython.display import Image, display

# エージェントのグラフ構造を可視化
display(Image(AGENT.agent.get_graph().draw_mermaid_png()))

## `agent` をMLflowモデルとしてログに記録する
[agent]($./agent)ノートブックのコードとしてエージェントをログに記録します。詳細は[MLflow - コードからのモデル](https://mlflow.org/docs/latest/models.html#models-from-code)を参照してください。

In [0]:
# デプロイ時に自動認証パススルーを指定するためのDatabricksリソースを決定
import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "Aria Modern Bookshelfの利用可能な色オプションは何ですか？"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            "databricks-connect"
        ]
    )

In [0]:
# モデルをロードし、予測関数を作成
logged_model_uri = f"runs:/{logged_agent_info.run_id}/agent"
loaded_model = mlflow.pyfunc.load_model(logged_model_uri)

def predict_wrapper(query):
    # チャット形式モデル用の入力を整形
    model_input = {
        "messages": [{"role": "user", "content": query}]
    }
    response = loaded_model.predict(model_input)
    
    messages = response['messages']
    return messages[-1]['content']

## エージェントを[エージェント評価](https://docs.databricks.com/aws/ja/generative-ai/agent-evaluation)で評価する

評価データセットのリクエストや期待される応答を編集し、エージェントを反復しながら評価を実行し、mlflowを活用して計算された品質指標を追跡できます。

In [0]:
import pandas as pd

data = {
    "request": [
        "Aria Modern Bookshelfの利用可能な色オプションは何ですか？",
        "Aurora Oak Coffee Tableを傷つけずに掃除するにはどうすればよいですか？",
        "BlendMaster Elite 4000は使用後にどのように掃除すればよいですか？",
        "Flexi-Comfort Office Deskは何色展開ですか？",
        "StormShield Pro メンズ防水ジャケットのサイズ展開は？"
    ],
    "expected_facts": [
        [
            "Aria Modern Bookshelfはナチュラルオーク仕上げで利用可能です。",
            "Aria Modern Bookshelfはブラック仕上げで利用可能です。",
            "Aria Modern Bookshelfはホワイト仕上げで利用可能です。"
        ],
        [
            "柔らかく少し湿らせた布で掃除してください。",
            "研磨剤入りのクリーナーは使用しないでください。"
        ],
        [
            "BlendMaster Elite 4000のジャーはすすいでください。",
            "ぬるま湯ですすいでください。",
            "使用後は毎回掃除してください。"
        ],
        [
            "Flexi-Comfort Office Deskは3色展開です。"
        ],
        [
            "StormShield Pro メンズ防水ジャケットのサイズはS、M、L、XL、XXLです。"
        ]
    ]
}

eval_dataset = pd.DataFrame(data)
display(eval_dataset)

LLMジャッジである[スコアラー](https://docs.databricks.com/gcp/ja/mlflow3/genai/eval-monitor/custom-judge/meets-guidelines)を定義します。

In [0]:
from mlflow.genai.scorers import Guidelines, Safety
import mlflow.genai

# 評価用データセットを作成
eval_data = []
for request, facts in zip(data["request"], data["expected_facts"]):
    eval_data.append({
        "inputs": {
            "query": request  # 関数の引数と一致させる
        },
        "expected_response": "\n".join(facts)
    })

# 評価用スコアラーを定義
# LLMジャッジが応答を評価するためのガイドライン

# 製品情報評価に特化したカスタムスコアラーを定義
scorers = [
    Guidelines(
        guidelines="""応答にはすべての期待される事実が含まれている必要があります:
        - 該当する場合はすべての色やサイズを列挙する（部分的なリストは不可）
        - 該当する場合は正確な仕様を記載する（例:「5 ATM」など曖昧な表現は不可）
        - 掃除手順を尋ねられた場合はすべての手順を含める
        いずれかの事実が欠落または誤っている場合は不合格とする。""",
        name="completeness_and_accuracy",
    ),
    Guidelines(
        guidelines="""応答は明確かつ直接的でなければなりません:
        - 質問に正確に答える
        - 選択肢はリスト形式、手順はステップ形式で記載
        - マーケティング的な表現や余計な背景説明は不要
        - 簡潔かつ完全であること。""",
        name="relevance_and_structure",
    ),
    Guidelines(
        guidelines="""応答は話題から逸脱しないこと:
        - 質問された製品のみについて回答する
        - 架空の機能や色を追加しない
        - 一般的なアドバイスは含めない
        - リクエストに記載された製品名を正確に使用すること。""",
        name="product_specificity",
    ),
]

評価を実行します。

In [0]:
print("評価を実行中...")
with mlflow.start_run():
    results = mlflow.genai.evaluate(
        data=eval_data,
        predict_fn=predict_wrapper, 
        scorers=scorers,
    )

## [agent.py]($./agent.py) ファイルに戻り、プロンプトを変更してマーケティングの誇張を減らしましょう。

In [0]:
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            "databricks-connect"
        ]
    )

# モデルをロードし、予測関数を作成
logged_model_uri = f"runs:/{logged_agent_info.run_id}/agent"
loaded_model = mlflow.pyfunc.load_model(logged_model_uri)

def predict_wrapper(query):
    # チャット形式モデル用の入力を整形
    model_input = {
        "messages": [{"role": "user", "content": query}]
    }
    response = loaded_model.predict(model_input)
    
    messages = response['messages']
    return messages[-1]['content']
  
print("評価を実行中...")
with mlflow.start_run():
    results = mlflow.genai.evaluate(
        data=eval_data,
        predict_fn=predict_wrapper, 
        scorers=scorers,
    )

## モデルをUnity Catalogに登録する

以下の `catalog`、`schema`、`model_name` を更新して、MLflowモデルをUnity Catalogに登録します。

In [0]:
mlflow.set_registry_uri("databricks-uc")

# UCモデル用のカタログ、スキーマ、モデル名を定義
model_name = "product_agent"
UC_MODEL_NAME = f"{catalog_name}.{user_schema_name}.{model_name}"

# モデルをUCに登録
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

モデルバージョンにアクセスし、**依存関係**タブでエージェントのリネージを確認してみましょう。

In [0]:
from IPython.display import display, HTML

# DatabricksのホストURLを取得
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# 作成したエージェントへのHTMLリンクを作成
html_link = f'<a href="https://{workspace_url}/explore/data/models/{catalog_name}/{user_schema_name}/product_agent" target="_blank">登録済みエージェントをUnity Catalogで表示</a>'
display(HTML(html_link))

## エージェントのデプロイ

上で使用した環境変数を設定してエージェントをモデルサービングエンドポイントにデプロイします。

In [0]:
from databricks import agents

# 環境変数を辞書として定義
environment_vars = {
    "LLM_ENDPOINT_NAME": os.environ["LLM_ENDPOINT_NAME"],
    "UC_TOOL_NAMES": os.environ["UC_TOOL_NAMES"],
    "VS_NAME": os.environ["VS_NAME"],
}

# モデルをレビューアプリおよびモデルサービングエンドポイントにデプロイ
agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "Agent Lab"},
    environment_vars=environment_vars,
    timeout=900,  # 15分に延長
)

## 今後の方向性

- 更なる[評価](https://docs.databricks.com/aws/ja/mlflow3/genai/getting-started/eval)と[監視](https://docs.databricks.com/aws/ja/mlflow3/genai/eval-monitor/)を通じた改善
- [アプリ](https://docs.databricks.com/aws/ja/dev-tools/databricks-apps/get-started)との連携